# [The StatQuest Illustrated Guide to Statistics](https://www.amazon.com/dp/B0GMP7Z9ZL)
## Chapter 05 - Quantifying Confidence in our Decisions with *p*-values!!!

Copyright 2026, Joshua Starmer

In this notebook we’ll learn how to...

- Calculate probabitlies with the **Binomal Distribution**.
- Calculate *p*-values with the **Binomial Distribution**.
- Observe an increase in False Positives when we use **One-Sided *p*-values**.

**NOTE:**
This tutorial assumes that you have installed **[R](https://cran.rstudio.com/)**, and possibly **[RStudio](https://posit.co/download/rstudio-desktop/)** and read Chapter 5 in **[The StatQuest Illustrated Guide to Statistics](https://www.amazon.com/dp/B0GMP7Z9ZL)**.

----

# Calculate probabilities with the **Binomial Distribution**

We'll start by calculating probabilities for coin tosses with the `dbinom()` function. In this case the `d` at the start of `dbinom()` stands for **Density** because statistical distributions are also called **Density Functions**. Personally, I'm just glad **Distribution** and **Density** both start with **D**, because otherwise I would never be able to remember that.

Anyway, `dbinom()` requires 3 parameters, `x`, the number of successes, `size`, the number of trials and `prob`, the probability of getting a success. In our example, `x` is the number of times we get heads, `size` is the number of times we flip the coin and `prob` is the probability that the coin will land on Heads, which, in our example, is 0.5.

So, to calculate the probability that we get 2 heads in 2 tosses, we use the follow function call:

In [ ]:
dbinom(x=2, size=2, prob=0.5)

To calculate the probability of getting 1 heads in 2 tosses, we use the following function call:

In [ ]:
dbinom(x=1, size=2, prob=0.5)

And to calculate the probability of getting 0 heads in 2 tosses, and getting 2 tails instead, we use the following function call:

In [ ]:
dbinom(x=0, size=2, prob=0.5)

**NOTE:** If we wanted to draw a histogram style graph that shows bars that reflect these probabilities, we can calculate all 3 probabilities at once by passing in the 3 ways we can get heads like this..

In [ ]:
## create a vector that has all of the
## outcomes we want to calculate probabilties for
x.vals <- c(2, 1, 0)

## calculate the probabilites for all of the
## values in x.vals
prob.vals <- dbinom(x=x.vals, size=2, prob=0.5)

## print out the probablities
prob.vals

Now we can use `x.vals` and `prob.vals` to draw the histogram like graph with the `barplot()` function. The `barplot()` can take a lot of arguments, but, for our example, we only need to specify the height of each bar with `height`, the names that go under each bar with `names.arg` and the overall label for the x-axis with `xlab`.

In [ ]:
barplot(height=prob.vals, # the height of each bar
        names.arg=x.vals, # the label under each bar
        xlab="Number of Heads") # the overall label for the x-axis

# Bam!

Now that we know how to calculate probabilities with the **Binomial Distribution** and draw a histogram like plot with those probabilities, let's learn how to calculate *p*-values with the **Binomial Distribution**.

-----

# Calculate *p*-values with the **Binomial Distribution**

To calculate *p*-values with the **Binomial Distribution** we can use the `binom.test()` function. Interestingly enough, the parameter names are not the same as they for `dbinom()`. Why this is, I have no idea. Specifically, instead of `size`, `binom.test` uses `n` to specify the number of trials. Anyway, in order to use `binom.test()`, we set `x`, the number of successes, `n`, the number of trials, and `p`, the probability of a success. In our example, `x` is the number of times we get Heads, `n` is the number of times we flip the coin, and `p` is the probability that the coin lands on heads.

So, if we wanted to calculate the *p*-value for getting 2 heads in 2 flips, we would use the following call to `binom.test()`.

In [ ]:
binom.test(x=2, n=2, p=0.5)

In the output, close to the top and to the right side, we can see the *p*-value, 0.5, which is what we calculated in the book.

Now let's calculate the *p*-value for getting 4 heads in 5 coin tosses...

In [ ]:
binom.test(x=4, n=5, p=0.5)

...and we get the same *p*-value, 0.375, that we got in the book (with rounding).

Lastly, let's calculate the *p*-value for getting 1 head in 10 coin tosses...

In [ ]:
binom.test(x=1, n=10, p=0.5)

...and we get the same *p*-value, 0.02, that we got in the book (with rounding).

## Double BAM!!

Now let's observe an increase in False Positives when we use One-Sided *p*-values.

----

# Observe an increase in False Positives when we use **One-Sided *p*-values**

To observe how a one-sided *p*-value can increase the number of false positives we report, we're going to let the data bias us. We'll use `rbinom()` to generate random numbers based on a **Binomial Distribution** where we flip a normal coin 100 times. These numbers will represent the number of heads we get in those 100 coin tosses. If that number of heads is greater than 50 (so, more heads than tails in 100 coin tosses), we'll test the hypothesis that our coin is normal with a one-sided *p*-value that only uses the side of the histogram that represents getting heads more frequently. Otherwise, we'll calculate a one-sided *p*-value that only uses the other side of the histogram that represents getting tails more frequently, or at least equally. If more than 5% of these tests are false positives, then we've observed an increase in False Positives.

We'll also calculate the two-sided *p*-values for each number of heads we get, so we can compare the results.

In [ ]:
## since we're going to generate random datasets,
## let's start by setting the seed so that the results
## are reproducable
set.seed(42)

## Next, we define the number of random
## datasets we wantt o create...
num.rand.datasets <- 10000

num.coin.flips <- 1000

## Create an empty arrays that are num.rand.datasets long
one.sided.p.values <- rep(NA, times=num.rand.datasets)
two.sided.p.values <- rep(NA, times=num.rand.datasets)

## Here is the loop were we create a bunch of random datasets,
## and then calculate a one-sided p-value for them, selecting
## the side that is more favorable to what we observed.
for(i in 1:num.rand.datasets) {
    
    ## First, figure out how many heads we got in num.coin.flips tosses
    num.heads <- rbinom(n=1, size=num.coin.flips, prob=0.5)

    ## Now calculate a two sided p-value for that
    results <- binom.test(x=num.heads, n=num.coin.flips, p=0.5)
    two.sided.p.values[i] <- results$p.value

    ## Now calculate a biased one-sided p-value
    ## that looks at the number of heads and decided
    ## which side to calculate the p-value with.
    if (num.heads > (num.coin.flips / 2)) {
        results <- binom.test(x=num.heads, n=num.coin.flips, p=0.5, alternative="greater") 
        one.sided.p.values[i] <- results$p.value
    } else {
        results <- binom.test(x=num.heads, n=num.coin.flips, p=0.5, alternative="less") 
        one.sided.p.values[i] <- results$p.value
    }
}

First, let's see how many false positives we get with the two-sided *p*-values.

In [ ]:
## First, determine the number of false positives with two-sided p-values
two.sided.num.false <- sum(two.sided.p.values < 0.05)

## print out the precentage
two.sided.num.false / num.rand.datasets

This is what we expected. With a threshold of 0.05, we expect about 5% of the time we'll get a false positive, and we got just a little bit less.

Now let's calculate the percentage of false positives we get with the one-sided *p*-values.

In [ ]:
## First, determine the number of false positives
one.sided.num.false <- sum(one.sided.p.values < 0.05)

## print out the precentage
one.sided.num.false / num.rand.datasets

Thus, 9% of the *p*-values were less than 0.05, twice as many as when we calculated two-sided *p*-values.

# BAM?

Well, at least we now know, first hand, that **One-Sided *p*-values** can be tricky and should be avoided unless we absolutely have to use them.

Bam.